In [1]:
from dnn.utils.dataLoader import dataLoader
from dnn.parameters import PARAM
from dnn.data.augmentation.detection import DetectionAugmentor
from dnn.data.augmentation.image import ImageAugmentor

import tensorflow as tf


The main issue now is the dataLoader does not run properly. Somehow we cannot iterate over parsed data.

In [2]:
# Apparently we can't iterate over our dataset
loader = dataLoader()
train = loader('F:/RM/iRM_Autonomy_2020/dnn/data/roco/ROCO_8.tfrecords')
train = train.shuffle(buffer_size = 512)
source = iter(train)
next(source)

InvalidArgumentError: Index out of range using input dim 1; input has only 1 dims
	 [[{{node StatefulPartitionedCall/strided_slice_9}}]]

In [3]:
# this is essentially taking apart our dataloader for easier debugging

FEATURES = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'class_n': tf.io.FixedLenFeature([], tf.string),
    'bbox_yxyx_n4': tf.io.FixedLenFeature([], tf.string),
}

detection_augmentor = DetectionAugmentor((416, 416))
image_augmentor = ImageAugmentor()

def _parse_example(tfrecord, aug = True):
    '''
    Takes tfrecord file, parse it, pass the data through augmentor, and return x_train, y_train
    Args:
        tfrecord: A tfrecord dataset
    '''
    example = tf.io.parse_single_example(tfrecord, FEATURES)

    augmentor_input = {
        'image_hw3':    tf.io.decode_image(example['image'], channels=3),
        'label_n':      tf.io.parse_tensor(example['class_n'], tf.int32),
        'bbox_yxyx_n4': tf.io.parse_tensor(example['bbox_yxyx_n4'], tf.float32),
    }

    # Manually set shape since it's not inferred
    augmentor_input['image_hw3'].set_shape((None, None, 3))
    augmentor_input['label_n'].set_shape((None,))
    augmentor_input['bbox_yxyx_n4'].set_shape((None, 4))
    if aug:
        # Run augmentation
        augmentor_output = detection_augmentor(augmentor_input)
        x_train_hw3 = augmentor_output['image_hw3']
        x_train_hw3 = image_augmentor(x_train_hw3)
        
        # Cat label behind
        y_train_n5 = tf.concat(
            [augmentor_output['bbox_yxyx_n4'] / 416, 
            tf.cast(tf.expand_dims(augmentor_output['label_n'], -1), tf.float32)], 
            axis = -1)
        
        return x_train_hw3, y_train_n5
    else:
        x_train_hw3 = tf.image.resize(augmentor_input['image_hw3'], (416, 416))
        y_train_n5 = tf.concat(
            [augmentor_input['bbox_yxyx_n4'] / 416,
            tf.expand_dims(tf.cast(augmentor_input['label_n'], tf.float32), -1)],
            axis= -1
        )
        return x_train_hw3, y_train_n5

In [4]:
#With out aug, we have shape issues
train = tf.data.TFRecordDataset('F:/RM/iRM_Autonomy_2020/dnn/data/roco/ROCO_8.tfrecords').map(lambda x : _parse_example(x, False))
for x, y in train.take(1):
    print(x)
    print(y)

InvalidArgumentError: ConcatOp : Ranks of all input tensors should match: shape[0] = [0] vs. shape[1] = [0,1]
	 [[{{node concat}}]]

In [5]:
# With aug, there's iterating issues
train = tf.data.TFRecordDataset('F:/RM/iRM_Autonomy_2020/dnn/data/roco/ROCO_8.tfrecords').map(lambda x : _parse_example(x, True))
for x, y in train.take(1):
    print(x)
    print(y)

InvalidArgumentError: Index out of range using input dim 1; input has only 1 dims
	 [[{{node StatefulPartitionedCall/strided_slice_8}}]]